# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4dd0aa1e20>
├── 'a' --> tensor([[-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870]])
└── 'x' --> <FastTreeValue 0x7f4dd0aa1eb0>
    └── 'c' --> tensor([[-0.0221, -1.2250, -1.2483, -1.4480],
                        [ 0.5378,  0.2966, -0.0364, -0.9326],
                        [-0.5676,  0.2178,  0.9872, -1.4102]])

In [4]:
t.a

tensor([[-0.5335,  0.5165, -1.1646],
        [ 0.7670, -0.4563, -1.2870]])

In [5]:
%timeit t.a

57.6 ns ± 0.0558 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4dd0aa1e20>
├── 'a' --> tensor([[-0.2869, -1.2739, -1.0894],
│                   [ 1.3295, -1.0510,  0.8349]])
└── 'x' --> <FastTreeValue 0x7f4dd0aa1eb0>
    └── 'c' --> tensor([[-0.0221, -1.2250, -1.2483, -1.4480],
                        [ 0.5378,  0.2966, -0.0364, -0.9326],
                        [-0.5676,  0.2178,  0.9872, -1.4102]])

In [7]:
%timeit t.a = new_value

60.3 ns ± 0.0508 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5335,  0.5165, -1.1646],
               [ 0.7670, -0.4563, -1.2870]]),
    x: Batch(
           c: tensor([[-0.0221, -1.2250, -1.2483, -1.4480],
                      [ 0.5378,  0.2966, -0.0364, -0.9326],
                      [-0.5676,  0.2178,  0.9872, -1.4102]]),
       ),
)

In [10]:
b.a

tensor([[-0.5335,  0.5165, -1.1646],
        [ 0.7670, -0.4563, -1.2870]])

In [11]:
%timeit b.a

59.5 ns ± 0.0475 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1303,  1.1465,  0.8921],
               [-0.7326,  0.3380, -0.7119]]),
    x: Batch(
           c: tensor([[-0.0221, -1.2250, -1.2483, -1.4480],
                      [ 0.5378,  0.2966, -0.0364, -0.9326],
                      [-0.5676,  0.2178,  0.9872, -1.4102]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.483 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

819 ns ± 0.202 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 403 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4dd0a920a0>
├── 'a' --> tensor([[[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]],
│           
│                   [[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]],
│           
│                   [[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]],
│           
│                   [[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]],
│           
│                   [[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]],
│           
│                   [[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]],
│           
│                   [[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]],
│           
│                   [[-0.5335,  0.5165, -1.1646],
│                    [ 0.7670, -0.4563, -1.2870]]])
└── 'x' --> <FastTreeValue 0x7f4d30ee9af0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 35.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4d07082940>
├── 'a' --> tensor([[-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870],
│                   [-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870],
│                   [-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870],
│                   [-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870],
│                   [-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870],
│                   [-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870],
│                   [-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870],
│                   [-0.5335,  0.5165, -1.1646],
│                   [ 0.7670, -0.4563, -1.2870]])
└── 'x' --> <FastTreeValue 0x7f4d07082640>
    └── 'c' --> tensor([[-0.0221, -1.2250, -1.2483, -1.4480],
                        [ 0.5378,  0.2966, -0.0364, -0.9326],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 45.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.7 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0221, -1.2250, -1.2483, -1.4480],
                       [ 0.5378,  0.2966, -0.0364, -0.9326],
                       [-0.5676,  0.2178,  0.9872, -1.4102]],
              
                      [[-0.0221, -1.2250, -1.2483, -1.4480],
                       [ 0.5378,  0.2966, -0.0364, -0.9326],
                       [-0.5676,  0.2178,  0.9872, -1.4102]],
              
                      [[-0.0221, -1.2250, -1.2483, -1.4480],
                       [ 0.5378,  0.2966, -0.0364, -0.9326],
                       [-0.5676,  0.2178,  0.9872, -1.4102]],
              
                      [[-0.0221, -1.2250, -1.2483, -1.4480],
                       [ 0.5378,  0.2966, -0.0364, -0.9326],
                       [-0.5676,  0.2178,  0.9872, -1.4102]],
              
                      [[-0.0221, -1.2250, -1.2483, -1.4480],
                       [ 0.5378,  0.2966, -0.0364, -0.9326],
                       [-0.5676,  0.2178,  0.9872, -1.4102]],

In [26]:
%timeit Batch.stack(batches)

79.7 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0221, -1.2250, -1.2483, -1.4480],
                      [ 0.5378,  0.2966, -0.0364, -0.9326],
                      [-0.5676,  0.2178,  0.9872, -1.4102],
                      [-0.0221, -1.2250, -1.2483, -1.4480],
                      [ 0.5378,  0.2966, -0.0364, -0.9326],
                      [-0.5676,  0.2178,  0.9872, -1.4102],
                      [-0.0221, -1.2250, -1.2483, -1.4480],
                      [ 0.5378,  0.2966, -0.0364, -0.9326],
                      [-0.5676,  0.2178,  0.9872, -1.4102],
                      [-0.0221, -1.2250, -1.2483, -1.4480],
                      [ 0.5378,  0.2966, -0.0364, -0.9326],
                      [-0.5676,  0.2178,  0.9872, -1.4102],
                      [-0.0221, -1.2250, -1.2483, -1.4480],
                      [ 0.5378,  0.2966, -0.0364, -0.9326],
                      [-0.5676,  0.2178,  0.9872, -1.4102],
                      [-0.0221, -1.2250, -1.2483, -1.4480],
                   

In [28]:
%timeit Batch.cat(batches)

148 µs ± 255 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 901 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
